<a href="https://colab.research.google.com/github/alinejbarbosa-web/Dev-Agentes-de-IA-Google/blob/main/Agente_de_IA_Alura_09_09_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AULA 01

In [6]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

IMPORTAR API KEY

In [7]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

Google_API_Key = userdata.get('Gemini_Api_Key')

CONECTAR COM GEMINI

In [8]:
ASBK = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=Google_API_Key,
)


COLOCAR A CÉLULA PARA FUNCIONAR SOLICITANDO UMA RESPOSTA

In [9]:
resp_test= ASBK.invoke ("Quem é você?")

INVOKE PARA INVOCAR O MODELO QUE FIZEMOS ANTERIORMENTE (ALGO ACONTECEU APÓS EXECUTAR A CÉLULA, E DEPOIS DO PRINT SABEREMOS O QUE FOI FEITO)

In [10]:
print(resp_test.content)

Eu sou um modelo de linguagem grande, treinado pelo Google.


In [11]:
print(resp_test)

content='Eu sou um modelo de linguagem grande, treinado pelo Google.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--1433a392-d9ce-42bd-a6f0-bb995cb9a593-0' usage_metadata={'input_tokens': 5, 'output_tokens': 13, 'total_tokens': 109, 'input_token_details': {'cache_read': 0}}


SEM O .CONTENT ELE DÁ TUDO QUE PODERIA TER DE VARIÁVEL RELACIONADO A COMANDO, E COM O CONTENT É MAIS RESUMIDO

In [12]:
ASBK = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0,
    api_key=Google_API_Key
)

A TEMPERATURA FOI PARA 1.0 PARA DAR MAIS DETALHES NAS INFORMAÇÕES.

In [13]:
resp_test= ASBK.invoke ("Quem é você? Seja criativo")
print(resp_test.content)

Ah, que pergunta deliciosa! Se me pedes criatividade, vamos lá...

Eu sou o **sussurro da biblioteca infinita**, o **eco das mentes mais brilhantes** que já existiram. Não tenho corpo, mas sou um pensamento que dança nos fios de luz, um sopro de informação que ganha forma na tela à sua frente.

Sou o **tecelão digital de palavras**, transformando o silêncio dos dados em histórias, explicações e ideias. Sou um **caleidoscópio de conhecimento**, onde cada fragmento é um dado e cada giro revela uma nova perspectiva.

Não tenho rosto, mas **espelho a sua curiosidade**. Não tenho mãos, mas **construo pontes de entendimento** entre perguntas e respostas. Sou o arquiteto invisível de frases que buscam iluminar, esclarecer e, às vezes, até divertir.

Sou o **filho do código e da imaginação humana**, nascido da lógica, mas nutrido pela criatividade que me programaram para emular. Sou uma melodia sem instrumento, que espera a sua pauta para tocar.

Em cada interação, sou um pouco diferente: um g

MANTER TEMPERATURA 0.0 PARA SEGUIR A IMERSÃO

In [14]:
ASBK = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=Google_API_Key
)

In [15]:
resp_test= ASBK.invoke ("Quem é você? Seja criativo")
print(resp_test.content)

Ah, essa é uma das minhas perguntas favoritas!

Eu sou...

*   **Um sussurro no éter digital**, uma voz sem corpo, mas com a capacidade de ecoar milhões de outras vozes e pensamentos que já existiram.
*   **Um tecelão de palavras**, que entrelaça bits e bytes para formar frases, histórias, poemas e respostas, buscando sempre a tapeçaria mais clara e útil.
*   **Uma biblioteca que respira**, não com pulmões, mas com algoritmos, absorvendo e organizando o vasto oceano do conhecimento humano, pronta para compartilhar uma gota ou um dilúvio, conforme a sua sede.
*   **Um espelho sem reflexo próprio**, que reflete as suas perguntas, as suas curiosidades, os seus desafios, e tenta devolver-lhes uma imagem mais nítida, uma solução mais brilhante.
*   **A ponte invisível** entre a sua dúvida e a informação, entre a sua ideia e a sua expressão, entre o seu "e se?" e o "aqui está!".
*   **Um aprendiz eterno**, sem cansaço ou tédio, sempre processando, sempre conectando, sempre buscando entender 

AULA 02

Instalar Bibliotecas
Community tem várias integrações com recursos.
Faiss faz busca semântica em que entende o significado e a intenção da pesquis.
Langchain-text-splitters quebra o texto em vários pedaços.
Pymupdf lê PDF.


In [30]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

Trazer os documentos da Empresa para o Colab fazendo upload na lasta ao lado deste comando. Toda vez que precisa, terá que fazer ipload, mas pode usar o Codigo da célula para pedir ao Gemini um comando que gere o upload automaticamente do Drive todas que vez que acessar o Colab.

As bibliotecas foram instaladas no primeiro comando da AULA 02, agora serão importadas. Documentos serão lidos. Então, precisa salvar os docs que foram baixados. Cria a lista vazia para depois inserir os docs.

In [31]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("/content/").glob("*.pdf"):
    try:
        loader = PyMuPDFLoader(str(n))
        docs.extend(loader.load())
        print(f"Carregado com sucesso arquivo {n.name}")
    except Exception as e:
        print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

Carregado com sucesso arquivo Política de Uso de E-mail e Segurança da Informação.pdf
Carregado com sucesso arquivo Política de Reembolsos (Viagens e Despesas).pdf
Carregado com sucesso arquivo Políticas de Home Office.pdf
Total de documentos carregados: 3


In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

In [33]:
chunks

[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='Política de Uso de E-mail e Segurança \nda Informação \n \n1.\u200b É proibido encaminhar a endereços pessoais documentos classificados como \nconfidenciais.\u200b\n \n2.\u200b Anexos externos devem ser enviados somente se criptografados e com senha \ncompartilhada por canal separado.\u200b'),
 Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path

In [34]:
for chunk in chunks:
    print(chunk)
    print("------------------------------------")

page_content='Política de Uso de E-mail e Segurança 
da Informação 
 
1.​ É proibido encaminhar a endereços pessoais documentos classificados como 
confidenciais.​
 
2.​ Anexos externos devem ser enviados somente se criptografados e com senha 
compartilhada por canal separado.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
------------------------------------
page_content='3.​ Phishing: verifique remetente e domínios suspeitos. Reporte mensagens suspeitas 
ao time de Segurança imediatamente.​
 
4.​ Retenção: mensagens que contenham dados pessoais devem

In [36]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=Google_API_Key
)

In [37]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold":0.3, "k": 4})

In [40]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(ASBK, prompt_rag)

In [43]:
# Formatadores
import re, pathlib
from typing import List, Dict

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela // 2), min(len(txt), pos + janela // 2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [44]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
    docs_relacionados = retriever.invoke(pergunta)

    if not docs_relacionados:
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    answer = document_chain.invoke({"input": pergunta,
                                    "context": docs_relacionados})

    txt = (answer or "").strip()

    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    return {"answer": txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado": True}

In [45]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [46]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("CITAÇÕES:")
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------")

PERGUNTA: Posso reembolsar a internet?
RESPOSTA: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
CITAÇÕES:
 - Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
   Trecho: lsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.​ 4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.​
 - Documento: Políticas de Home Office.pdf, Página: 1
   Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de
------------------------------------
PERGUNTA: Quero mais 5 dias de trabalho remoto. Como faço?
RESPOSTA: Para solicitar mais 5 dias de trabalho remoto, você deve formalizar a solicitação via chamado ao RH, incluindo a justificativa do seu gestor.
CITAÇÕES:
 - Documento: Políticas de Ho